In [ ]:
!pip install nltk

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import os

In [ ]:
# Read the CSV file into a Pandas DataFrame
data = pd.read_csv('/kaggle/input/memexdata/images_info.csv')
data.head()

In [ ]:
import pandas as pd

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv('/kaggle/input/memexdata/images_info.csv')

# Splitting the 'image_path' column to extract filenames
data['image_path'] = data['image_path'].apply(lambda x: x.split('/')[-1].split('.')[0])  # Extracts the filename without extension
data['sentences'] = data['sentences']  # Example assignment, you might have different data to put here
data['Meme_Caption'] = data['Meme_Caption']  # Example assignment, you might have different data to put here
data['Relevance_Score'] = data['Relevance_Score']  # Example assignment, you might have different data to put here

# Save the modified DataFrame to a new CSV file
data.to_csv('/kaggle/working/modified_images_info.csv', index=False)


In [ ]:
# Read the CSV file into a Pandas DataFrame
data = pd.read_csv('/kaggle/working/modified_images_info.csv')
data.head()

In [ ]:
import pandas as pd
import os

# Path to your image folder
image_folder_path = '/kaggle/input/memexdata/Meme/Meme/images'

# Load the CSV file into a Pandas DataFrame
data = pd.read_csv('/kaggle/working/modified_images_info.csv')

# Create lists to store image paths and information
image_paths = []
sentences_list = []
evidence_list = []
meme_captions = []
relevance_scores = []

# Iterate through the CSV data and map images from the folder
for index, row in data.iterrows():
    image_name = str(row['image_path']) + '.jpg'  # Assuming the image paths in CSV are numbers without extension
    image_path = os.path.join(image_folder_path, image_name)
    
    # Check if the image exists in the folder
    if os.path.exists(image_path):
        image_paths.append(image_path)
        sentences_list.append(row['sentences'])
        evidence_list.append(row['evidence'])
        meme_captions.append(row['Meme_Caption'])
        relevance_scores.append(row['Relevance_Score'])

# Create a DataFrame from the mapped information
image_info = pd.DataFrame({
    'image_path': image_paths,
    'sentences': sentences_list,
    'evidence': evidence_list,
    'Meme_Caption': meme_captions,
    'Relevance_Score': relevance_scores
})

# Save the DataFrame to a CSV file
image_info.to_csv('/kaggle/working/images_info_mapped.csv', index=False)


In [ ]:
# Read the CSV file into a Pandas DataFrame
data = pd.read_csv('/kaggle/working/images_info_mapped.csv')
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
# Splitting the data into train and temp sets (80% train, 20% temp)
train_df, temp_df = train_test_split(data, test_size=0.2, random_state=42)

# Splitting the temp data into test and validation sets (50% test, 50% validation from the temp data)
test_df, validation_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Display the sizes of each split
print(f"Train dataset size: {len(train_df)}")
print(f"Test dataset size: {len(test_df)}")
print(f"Validation dataset size: {len(validation_df)}")

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from transformers import BertTokenizer, BertModel, AdamW
import torchvision.models as models
from tqdm import tqdm

In [ ]:
!pip install --upgrade transformers

In [ ]:
from transformers import AutoImageProcessor, SwiftFormerModel
import torch
image_processor = AutoImageProcessor.from_pretrained("MBZUAI/swiftformer-xs")
model = SwiftFormerModel.from_pretrained("MBZUAI/swiftformer-xs")

In [ ]:
from transformers import AutoTokenizer, XLMRobertaModel, AdamW
# Initialize BERT tokenizer and model
bert_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
bert_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

In [ ]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
model.to(device)

In [ ]:
bert_model.to(device)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torchvision.models as models

In [ ]:
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset


max_seq_length = 512  # Set your desired maximum sequence length for BERT

# Define the pre-processing transformations for images
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class MyMultimodalDataset(Dataset):
    def __init__(self, data, transform=None, tokenizer=None, max_seq_length=128):
        self.data = data
        self.transform = transform
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]['image_path']
        image = Image.open(image_path).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        sentences = self.data.iloc[idx]['sentences']
        evidence = self.data.iloc[idx]['evidence']
        meme_caption = self.data.iloc[idx]['Meme_Caption']

        # Concatenate all text inputs into a single string
        combined_text = f"{sentences} {evidence} {meme_caption}"

        inputs = self.tokenizer(combined_text, padding='max_length', truncation=True, max_length=self.max_seq_length, return_tensors='pt')
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        label = self.data.iloc[idx]['Relevance_Score']

        return image, input_ids, attention_mask, label


In [ ]:
# Create custom datasets with MyMultimodalDataset
train_dataset = MyMultimodalDataset(train_df, transform=transform, tokenizer=bert_tokenizer, max_seq_length=max_seq_length)
test_dataset = MyMultimodalDataset(test_df, transform=transform, tokenizer=bert_tokenizer, max_seq_length=max_seq_length)
val_dataset = MyMultimodalDataset(validation_df, transform=transform, tokenizer=bert_tokenizer, max_seq_length=max_seq_length)

# Define data loaders
batch_size = 5  # Set your desired batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch
import time
from torch.optim import AdamW
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
# Define optimizer and loss function
optimizer = AdamW(list(model.parameters()) + list(bert_model.parameters()), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
import torch
import torchvision.models as models
from transformers import AutoImageProcessor, ViTModel, AutoTokenizer, DistilBertModel, AdamW
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Assuming you have defined your train_loader, val_loader, optimizer, criterion, model, bert_model, etc.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 1
train_losses = []
val_losses = []

# Training time
start_time = time.time()

for epoch in range(num_epochs):
    running_train_loss = 0.0
    
    model.train()
    bert_model.train()

    for images, texts, attention_masks, labels in train_loader:
        images = images.to(device)
        labels = (labels - 1).to(device)

        input_ids = texts.squeeze(1).to(device)
        attention_mask = attention_masks.squeeze(1).to(device)

        optimizer.zero_grad()

        with torch.no_grad():
            outputs_image = model(pixel_values=images)
        img_hidden_states = outputs_image.last_hidden_state
        img_feats = img_hidden_states[:, 0, :]

        outputs_text = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        text_hidden_states = outputs_text.last_hidden_state
        text_feats = text_hidden_states[:, 0, :]

        # Check and reshape tensors if needed for compatibility
        if img_feats.size(-1) != text_feats.size(-1):
            # Assuming img_feats and text_feats have the same first dimension (batch_size)
            img_feats = img_feats.view(img_feats.size(0), -1)  # Reshape img_feats if necessary
            text_feats = text_feats.view(text_feats.size(0), -1)  # Reshape text_feats if necessary

        combined_feats = torch.cat((img_feats, text_feats), dim=1)

        regressor = torch.nn.Sequential(
            torch.nn.Linear(combined_feats.shape[1], 512).to(device),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(512, 5).to(device),  # Change output size to 5 for 5 labels
        )

        predictions = regressor(combined_feats)
        loss = criterion(predictions, labels)

        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    epoch_train_loss = running_train_loss / len(train_loader)
    train_losses.append(epoch_train_loss)

    model.eval()
    bert_model.eval()

    running_val_loss = 0.0

    with torch.no_grad():
        for val_images, val_texts, val_attention_masks, val_labels in val_loader:
            val_images = val_images.to(device)
            val_labels = (val_labels - 1).to(device)

            val_input_ids = val_texts.squeeze(1).to(device)
            val_attention_mask = val_attention_masks.squeeze(1).to(device)

            outputs_image = model(pixel_values=val_images)
            val_img_hidden_states = outputs_image.last_hidden_state
            val_img_feats = val_img_hidden_states[:, 0, :]

            outputs_text = bert_model(input_ids=val_input_ids, attention_mask=val_attention_mask)
            val_text_hidden_states = outputs_text.last_hidden_state
            val_text_feats = val_text_hidden_states[:, 0, :]

            # Check and reshape tensors if needed for compatibility in validation set
            if val_img_feats.size(-1) != val_text_feats.size(-1):
                val_img_feats = val_img_feats.view(val_img_feats.size(0), -1)
                val_text_feats = val_text_feats.view(val_text_feats.size(0), -1)

            val_combined_feats = torch.cat((val_img_feats, val_text_feats), dim=1)

            val_predictions = regressor(val_combined_feats)
            val_loss = criterion(val_predictions, val_labels)

            running_val_loss += val_loss.item()

    epoch_val_loss = running_val_loss / len(val_loader)
    val_losses.append(epoch_val_loss)

    print(f"Epoch [{epoch + 1}/{num_epochs}] - "
          f"Train Loss: {epoch_train_loss:.4f}, "
          f"Val Loss: {epoch_val_loss:.4f}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Total execution time: {execution_time:.2f} seconds")

In [ ]:
import torch
from tqdm import tqdm
import time

# Set models to evaluation mode
model.eval()
bert_model.eval()

# Prepare lists to store predicted and true labels
predicted_labels = []
true_labels = []

# Set your device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Test loop
start_time = time.time()

with torch.no_grad():
    for test_images, test_texts, test_attention_masks, test_labels in tqdm(test_loader, desc='Testing', leave=False):
        test_images = test_images.to(device)
        test_labels = (test_labels - 1).to(device)  # Convert labels from 1-5 to 0-4
        test_texts = test_texts.to(device)
        test_attention_masks = test_attention_masks.to(device)

        optimizer.zero_grad()

        # Extract features from image-based model
        test_img_feats = model(test_images)

        # Extract features from text-based model (BERT)
        test_texts = test_texts.squeeze(1)
        test_attention_masks = test_attention_masks.squeeze(1)
        test_outputs = bert_model(input_ids=test_texts, attention_mask=test_attention_masks)
        
        # Extract relevant features for concatenation
        # Handle tensor manipulation for compatibility
        # Reshape test_img_feats if compatible
        test_img_feats = test_img_feats[0]  # Access a specific part of the BaseModelOutputWithNoAttention
        test_img_feats = test_img_feats.reshape(test_img_feats.shape[0], -1)  # Reshape if compatible

        # Extract representations from text-based model
        test_text_feats = test_outputs.last_hidden_state[:, 0, :]  # Select appropriate representations

        # Combine features early
        combined_feats = torch.cat((test_img_feats, test_text_feats), dim=1)

        # Classify combined features
        combined_classifier = torch.nn.Sequential(
            torch.nn.Linear(combined_feats.shape[1], 512).to(device),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(512, 5).to(device),  # Change the output size to 5 for 5 labels
        )

        combined_logits = combined_classifier(combined_feats)
        test_predictions = torch.nn.functional.softmax(combined_logits, dim=1)
        predicted_classes = torch.argmax(test_predictions, dim=1) + 1  # Revert back to labels from 1-5, Add 1 here 

        predicted_labels.extend(predicted_classes.cpu().numpy())
        true_labels.extend(test_labels.cpu().numpy().tolist())

end_time = time.time()
execution_time = end_time - start_time

# Print or use the predicted labels and true labels as needed
print("Predicted Labels:", predicted_labels)
print("True Labels:", true_labels)
print(f"Total execution time for testing: {execution_time:.2f} seconds")


In [ ]:
predicted_labels 

In [ ]:
true_labels 

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, cohen_kappa_score, mean_squared_error, classification_report, roc_auc_score, hamming_loss


# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate precision, recall, F1-score overall (macro average)
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Calculate Mean Squared Error
mse = mean_squared_error(true_labels, predicted_labels)

# Print overall calculated metrics
print(f"Accuracy: {accuracy}")
print(f"Precision (macro): {precision}")
print(f"Recall (macro): {recall}")
print(f"F1-Score (macro): {f1_score}")
print(f"Mean Squared Error: {mse}")

In [ ]:
# Generate classification report
classification_rep = classification_report(true_labels, predicted_labels)
print(f"Classification Report:\n{classification_rep}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

# Assuming conf_matrix is your confusion matrix with shape (n_classes, n_classes)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Set font properties
font = {'family': 'serif', 'weight': 'bold', 'size': 12}
font2 = {'family': 'serif', 'weight': 'bold', 'size': 14}

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)*100

# Create a heatmap using the 'Paired' color palette without color bar
plt.figure(figsize=(5, 4))
sns.heatmap(conf_matrix, annot=True, cmap="crest", fmt='g', cbar=False)

plt.xlabel('Predicted Labels', labelpad=12, fontdict=font)
plt.ylabel('True Labels', labelpad=12, fontdict=font)
plt.title(f'Relevance Score\nAccuracy: {accuracy:.2f}%', pad=12, fontdict=font2)

plt.show()
